## Arakawa C网格的差分和插值
**05 Febrary 2020 by MiniUFO**

---
[TOC]

---

### 1. 问题
2020年开年有点糟心，我还在家里躲避疫情。闲来没事想证明一个东西，就是`Arakawa C`网格的差分后再插值，等于`Arakawa A`网格的中央差分。咳咳，我是谁，我在哪，我要干啥...

![Arakawa grids](https://upload.wikimedia.org/wikipedia/commons/8/8a/Discretization_for_the_different_Arakawa_grids_correct.svg)
**<center>图1. Arakawa A、B、C、D、E网格示意图</center>**

以前自己编写差分程序都是用中央差分，原因是截断误差比前差、后差小，比如：
>$$\frac{du}{dx}=\left(\frac{\Delta u}{\Delta x}\right)_{i}=\frac{u_{i+1}-u_{i-1}}{2\Delta x} \tag{1}$$

这样可以保证$(du/dx)_i$和$u_i$定义在同一个网格上。

另外，我也很早就发现`Matlab`中有`diff`函数（当然`Python`的`Numpy`中也有），但是`diff`的结果是比原来的数组少一个元素，也即`N`个相邻元素的差值，结果就只有`N-1`个。当时觉得这就是前差或者后差，误差大，并且结果定义在哪个网格点不够明确。后来我慢慢明白，如果把结果定义在半个网格上，那就容易理解很多了，也是中央差分，只不过结果与原数据错开了半个网格：
>$$\left(\frac{\Delta u}{\Delta x}\right)_{i+0.5}=\frac{u_{i+1}-u_{i}}{\Delta x} \tag{2}$$

Arakawa C网格在模式中的广泛使用让我慢慢明白，这种策略已经被科学界普遍接受，但也有一些麻烦，比如变量和变量的导数定义会错开半个网格，不同变量的位置定义不完全一致。举个例子，动量$u$、$v$定义在不同位置，导致动能$(u^2+v^2)/2$和它们也错位，计算动能需要插值$u$、$v$到相同网格上再进行。

我要做个人狠话不多的博主...我的目标就是证明：
>$$\left(\frac{\Delta u}{\Delta x}\right)_i=\frac{1}{2}\left[\left(\frac{\Delta u}{\Delta x}\right)_{i-0.5}+\left(\frac{\Delta u}{\Delta x}\right)_{i+0.5}\right] \tag{3}$$

### 2. 证明
有人一眼就看出来左右相等了？好吧，我承认我的反应有点慢。其实`(3)`式左边是我用以前的`Java`程序实现的中央差，右边则可以用`Python`的`diff`和`interp`完成。想要证明两个方法产生一样的结果，先要理论上证明它们相等。
>$$(3)式左边=\left(\frac{\Delta u}{\Delta x}\right)_i=\frac{u_{i+1}-u_{i-1}}{x_{i+1}-x_{i-1}}=\frac{u_{i+1}-u_{i-1}}{2\Delta x} \tag{5}$$

>$$(3)式右边=\frac{1}{2}\left[\left(\frac{\Delta u}{\Delta x}\right)_{i-0.5}+\left(\frac{\Delta u}{\Delta x}\right)_{i+0.5}\right]=\frac{1}{2}\left[\frac{u_{i}-u_{i-1}}{\Delta x}+\frac{u_{i+1}-u_{i}}{\Delta x}\right] \tag{6}$$

好吧，这时候我也看出来相等了。左边用`Java`的计算代码是：

```Java
Variable u = ...;
Variable dudx = u.cDerivative(Dimension.X)
```

右边用`Python`（`xarray`）的计算代码是：
```Python
# method 1
u = ...
dudx_l = grid.derivative(u, 'X')
dudx  = grid.interp(dudx_1, 'X')

# method 2
u = ...
ul = grid.interp(u, 'X')
dudx = grid.derivative(u, 'X')
```

事实怎样以后再弄吧，有点累了，吃点零食先。

#### References:
